<a href="https://colab.research.google.com/github/RamizAlhaj/interface_project/blob/main/Copy_of_Copy_of_Untitled3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
import string
import openpyxl as xl 
import re

In [ ]:
#load the data set we need
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (4).csv


In [ ]:
#read the exel file
dataSet = pandas.read_csv('train.csv')
#print the first 5 rows of the data
dataSet.head(5)

,Id,Tweet,following,followers,actions,is_retweet,location,Type
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,NaN,0.0,Chicago,Quality
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,NaN,0.0,NaN,Quality
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,NaN,0.0,"Scotland, U.K",Quality
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",Spam
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,Spam


In [ ]:
#print the shape ( get the number of rows and colums) 
dataSet.shape

(11968, 8)

In [ ]:
#get the columnes name
dataSet.columns

Index(['Id', 'Tweet', 'following', 'followers', 'actions', 'is_retweet',
       'location', 'Type'],
      dtype='object')

In [ ]:
# show the new shape if any dublicate tweet removed
dataSet.shape

(11968, 8)

In [ ]:
#show the number of missing data for each column
dataSet.isnull().sum()

Id               0
Tweet            0
following      145
followers       16
actions       2773
is_retweet       1
location      1651
Type             0
dtype: int64

In [ ]:
#upload the feature vector file's
from google.colab import files
uploaded1 = files.upload()

In [ ]:
#read the file of feature vector
with open('featureVector.txt') as feature:
    featureVector = feature.readlines()
    print(featureVector)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
#check for the duplicates tweet and remove them                                                    
duplicate = dataSet[dataSet.duplicated(subset=["Tweet"], keep="first")]
data = []
for i in range(dataSet.shape[0]):
  dataReview = dataSet.iloc[i, 1]
  #remove non alphabetic characters
  dataReview = re.sub('[^A-Za-z]', ' ', dataReview)
  #make all words in lowercase
  dataReview = dataReview.lower()
  #tokenization
  tokenized_dataReview = word_tokenize(dataReview)
  #removing stopwords
  dataReview_processed = []
  for word in tokenized_dataReview:
    if word not in set(stopwords.words('english')):
      #stemming
      dataReview_processed.append(stemmer.stem(word))
  tweet_review = " ".join(dataReview_processed)
  data.append(tweet_review)


In [ ]:
print(data)

In [ ]:

def process_text(Tweet):
  with open('train.csv') as dat, open('featureVector.txt') as feature:
    features = set(feature.read().splitlines())
    # process_text1(Tweet)
   
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    #check for the duplicates tweet and remove them  
    duplicate = dataSet[dataSet.duplicated(subset=["Tweet"], keep="first")]
   
    quality = []
    spam = []
    words = Tweet.split()
    hashtage = (r'\b[#]\w+')
    found = False
    for word in words:
      if word in features:
        spam.append(Tweet)
        found = True
        break
      if not found:
        quality.append(Tweet)
    print(len(spam), len(quality))
   
  
  return quality

In [ ]:
# show the tokenization (a list of tokens)
dataSet['Tweet'].head().apply(process_text)

In [ ]:
#this fuction to show if check the spam of not !
def is_spam(tweet):                                  
    with open('train.csv') as dat, open('featureVector.txt') as feature:
        #next(dataSet, None)
        features = set(feature.read().splitlines())
        quality = []
        spam = []
        words = tweet.split()
        for word in words:
          if word in features:
            return True
        return False     

In [ ]:
# show the tokenization (a list of tokens)
output=dataSet['Tweet'].apply(is_spam)
print("we fight")
print(output)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=process_text).fit_transform(dataSet['Tweet'])

In [ ]:
print(message_bow.shape)

In [ ]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(message_bow , dataSet['Type'], test_size=0.20, random_state = 0)

In [ ]:
#get the shape of the data 
message_bow.shape

In [ ]:
#creat and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(x_train, y_train)

In [ ]:
#print the predictions
print(classifier.predict(x_train))
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_train.shape)
#print the actual values
print(y_train.values)

In [ ]:
#evalueate the model on the training data set 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(x_train)
print()
print(classification_report(y_train,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_train, pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

In [ ]:
#print the predictions
print(classifier.predict(x_test))

#print the actual values
print(y_test.values)

In [ ]:
#creat and train the naive bayes classifier                                      
from sklearn.naive_bayes import MultinomialNB
classifier2 = MultinomialNB().fit(x_test, y_test)

In [ ]:
#evalueate the model on the testing data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier2.predict(x_test)
print(classification_report(y_test,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_test, pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

In [ ]:
#creat and train the naive bayes classifier                                     
from sklearn.neural_network import MLPClassifier

classifier3 = MLPClassifier().fit(x_test, y_test)

In [ ]:
#evalueate the model on the testing data set using nural netwerk
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from numpy import mean
pred = classifier3.predict(x_test)
print(classification_report(y_test,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_test, pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

print('Mean: %.3f' %mean(accuracy_score(y_test,pred)))


In [ ]:
#creat and train the naive bayes classifier                                      
from sklearn.neural_network import MLPClassifier

classifier31 = MLPClassifier().fit(x_train, y_train)


In [ ]:
#evalueate the model on the training data set using nural netwerk
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier31.predict(x_train)
print(classification_report(y_train,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_train, pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))
print('Mean: %.3f' %mean(accuracy_score(y_train,pred)))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
est = DecisionTreeClassifier().fit(x_train, y_train)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from numpy import mean
pred = est.predict(x_test)
print(classification_report(y_test,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_test, pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

print('Mean: %.3f' %mean(accuracy_score(y_test,pred)))
# pred = classifier31.predict(x_test)
# print(classification_report(y_test,pred))
# print()
# print('confusion matrix: \n', confusion_matrix(y_test, pred))
# print()
# print('Accuracy: ', accuracy_score(y_test,pred))

# print('Mean: %.3f' %mean(accuracy_score(y_test,pred)))

In [ ]:

# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# from numpy import mean
# pred = est.predict(x_train)
# print(classification_report(y_train,pred))
# print()
# print('confusion matrix: \n', confusion_matrix(y_train, pred))
# print()
# print('Accuracy: ', accuracy_score(y_train,pred))

# print('Mean: %.3f' %mean(accuracy_score(y_train,pred)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
g = DecisionTreeClassifier().fit(x_train, y_train)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from numpy import mean
pred = g.predict(x_train)
print(classification_report(y_train,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_train, pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

print('Mean: %.3f' %mean(accuracy_score(y_train,pred)))

In [ ]:
# dpdpdpdpdpdpd
from sklearn.tree import DecisionTreeClassifier
m = DecisionTreeClassifier().fit(x_test, y_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from numpy import mean
pred = m.predict(x_train)
print(classification_report(y_train,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_train, pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

print('Mean: %.3f' %mean(accuracy_score(y_train,pred)))


In [ ]:
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from numpy import mean
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
# classifier4 = DecisionTreeClassifier().fit(x_test, y_test)
est = DecisionTreeClassifier().fit(x_train, y_train)
pred = est.predict(x_test)
# print(classification_report(y_test,pred))
# print()
# print('confusion matrix: \n', confusion_matrix(y_test, pred))
# print()
# print('Accuracy: ', accuracy_score(y_test,pred))

# print('Mean: %.3f' %mean(accuracy_score(y_test,pred)))
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# x, y = make_classification(n_samples=100, random_state=1)
# x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y,                                                    random_state=1)
# clf = est.fit(x_train, y_train)
# clf1 = classifier4.fit(x_test, y_test)
# clf.predict_proba(x_train)
# clf1.predict(x_test)
# c1 = clf1.score(x_test, y_test)
# c2 = clf.score(x_train, y_train)
# print(" Test dataSet :",c1,"\n","Train dataSet :",c2)
clf = tree.DecisionTreeClassifier(random_state=0)
# tree.plot_tree(decision_tree= tree,fontsize=40)
iris = load_iris()
clf = clf.fit(iris.data, iris.target)
# clf = plt.figure(figsize=(200,250))
tree.plot_tree(clf)
# print ("\n")
# clf = plt.figure(figsize=(200,250))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):

    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        x_test,
        y_train,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="y", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

x_test, y_train = load_digits(return_X_y=True)

title = "Learning Curves (Naive Bayes)"
# Cross validation with 50 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=1000, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(
    estimator, title, x_test, y_train, axes=axes[:, 0], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(
    estimator, title, x_train, y_train, axes=axes[:, 1], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)

plt.show()

In [ ]:

import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_gaussian_quantiles

X,y = make_gaussian_quantiles(n_samples = 11969, n_features = 300 , n_classes= 2 , random_state =1 )
df = pd.DataFrame(dict(y_train=X[:,0], y_test = X[:,1],label=y))
plt.scatter(X[:,0],X[:,1], marker='o',c=y,s=25,edgecolors='b')

In [ ]:
from  sklearn.model_selection import  train_test_split
target = df.label
df.drop(['label'], axis = 1, inplace=True)
y = target
x = df
message_bow = CountVectorizer(analyzer=process_text).fit_transform(dataSet['Tweet'])

X_train , X_val,Y_train,Y_val = train_test_split(x,y, test_size=  0.20, random_state=1)
X_train.shape  , X_val.shape,Y_train.shape,Y_val.shape

In [ ]:
from  sklearn.tree import  DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=1,min_samples_split=50,criterion='entropy')


In [ ]:
path = model.cost_complexity_pruning_path(X_train,Y_train)
path

In [ ]:
ccp_alphas , impurities = path.ccp_alphas , path.impurities
plt.figure(figsize=(10,6))
plt.plot(ccp_alphas,impurities)